In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install pingouin

In [ ]:
import numpy as np

EPSILON = 1e-10


def rae(actual: np.ndarray, predicted: np.ndarray):
    """ Relative Absolute Error (aka Approximation Error) """
    return np.sum(np.abs(actual - predicted)) / (np.sum(np.abs(actual - np.mean(actual))) + EPSILON)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv('anon_jobs_final.csv')

df['QueueID'] = df['QueueID'].str[1:].astype(int)
df['ExecutableID'] = df['ExecutableID'].str[1:].astype(int)
df['GroupID'] = df['GroupID'].str[1:].astype(int)

historical_data=df[["AverageCPUTimeUsed","SubmitTime","UserID","UsedMemory","ReqMemory","Status","WaitTime","ReqTime","ReqMemory","PartitionID","Nprocs","ExecutableID","ReqNProcs","OrigSiteID","LastRunSiteID","GroupID","QueueID"]]
target=df[["RunTime"]]

le = LabelEncoder()
le.fit(historical_data['UserID'].astype(str))
historical_data['UserID'] = le.transform(historical_data['UserID'].astype(str))

le.fit(historical_data['OrigSiteID'].astype(str))
historical_data['OrigSiteID'] = le.transform(historical_data['OrigSiteID'].astype(str))

le.fit(historical_data['LastRunSiteID'].astype(str))
historical_data['LastRunSiteID'] = le.transform(historical_data['LastRunSiteID'].astype(str))

X_train, X_test, y_train, y_test = train_test_split(historical_data, target, test_size=0.33, random_state=42)


In [ ]:
y_train=y_train.values.reshape(-1)

In [ ]:
y_test=y_test.values.reshape(-1)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X_train)
X_testscaled=sc_X.transform(X_test)

MLP

In [ ]:
from sklearn.neural_network import MLPRegressor
reg = MLPRegressor(hidden_layer_sizes=(128,128,128),activation="relu" ,random_state=1, max_iter=2000).fit(X_trainscaled, y_train)

In [ ]:
y_pred=reg.predict(X_testscaled)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)yhh

In [ ]:
rae(y_test,y_pred)

XGBOOST

In [ ]:
import xgboost as xgb_r

In [ ]:
xgb_r = xg.XGBRegressor()
xgb_r.fit(X_train,y_train)

In [ ]:
pred=xgb_r.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,pred)

In [ ]:
rae(y_test,pred)

KNN (N=3)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                    weights='uniform')

In [ ]:
y_pred=knn.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.914773,"[0.91, 0.92]",0.83681,0.836807,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.09044813912876593

KNN (N=5)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred=knn.predict(X_test)


In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.919675,"[0.92, 0.92]",0.845802,0.845799,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.09749527494371557

KNN (N=7)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=7)
knn.fit(X_train, y_train)
y_pred=knn.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.919912,"[0.92, 0.92]",0.846239,0.846236,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.10456838650352898

KNN N=9

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=9)
knn.fit(X_train, y_train)
y_pred=knn.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.919968,"[0.92, 0.92]",0.846342,0.846339,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.11117160527509375

RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train, y_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

In [ ]:
rae(y_test,y_pred)

DECISION TREES

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
import pingouin as pg
pg.corr(y_test,y_pred)

,n,r,CI95%,r2,adj_r2,p-val,BF10,power
pearson,114712,0.920973,"[0.92, 0.92]",0.848191,0.848188,0.0,inf,1.0


In [ ]:
rae(y_test,y_pred)

0.048129290921626613